In [ ]:
 Streamlit video upload
uploaded_video = st.file_uploader("Upload a video", type=["mp4", "mov", "avi", "mkv"])


In [ ]:
pip install torch torchvision transformers pillow

In [ ]:
import os
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [ ]:
# Streamlit app title
st.title("Video Summarization App")

2024-10-16 04:46:24.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-16 04:46:24.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
frames_dir = "/content/drive/MyDrive/unique_frames"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
pip install pillow

In [ ]:
def generate_caption(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

In [ ]:
# Loop through all the unique frames and generate captions
captions = {}
for frame_filename in os.listdir(frames_dir):
    if frame_filename.endswith('.jpg') or frame_filename.endswith('.png'):
        frame_path = os.path.join(frames_dir, frame_filename)
        caption = generate_caption(frame_path)
        captions[frame_filename] = caption
        print(f"Caption for {frame_filename}: {caption}")

Caption for frame_0.jpg: a car is driving down the road near a traffic light
Caption for frame_3.jpg: a street with a lot of traffic on it
Caption for frame_1.jpg: a street with a bunch of cars on it
Caption for frame_2.jpg: a street with a lot of traffic on it
Caption for frame_4.jpg: a street with a bunch of cars on it
Caption for frame_5.jpg: a street with a bunch of cars on it
Caption for frame_6.jpg: a view of a street with a lot of traffic
Caption for frame_7.jpg: a view of a street with cars and a bus
Caption for frame_8.jpg: a street with cars and a sign on it
Caption for frame_10.jpg: a view of a street with a bunch of cars
Caption for frame_9.jpg: a street with a bunch of cars on it
Caption for frame_11.jpg: a view of a street with a bunch of cars
Caption for frame_12.jpg: a view of a street with a bus on it
Caption for frame_13.jpg: a street with a bunch of cars on it
Caption for frame_15.jpg: a street with a bunch of cars on it
Caption for frame_17.jpg: a street with a bunc

In [ ]:
with open('captions.txt', 'w') as f:
    for frame_filename, caption in captions.items():
        f.write(f"{frame_filename}: {caption}\n")

In [ ]:
from difflib import SequenceMatcher

# Function to load captions from the captions.txt file
def load_captions(captions_file):
    with open(captions_file, 'r') as f:
        captions = f.readlines()
    return [caption.strip() for caption in captions]

# Function to check similarity between two strings
def are_similar(caption1, caption2, threshold=0.85):
    similarity_ratio = SequenceMatcher(None, caption1, caption2).ratio()
    return similarity_ratio > threshold

# Function to create a paragraph summary from all captions, removing duplicates or similar captions
def create_paragraph_summary(captions, similarity_threshold=0.85):
    unique_captions = []

    # Loop through captions and add only unique or dissimilar ones
    for i, caption in enumerate(captions):
        if not any(are_similar(caption.split(': ')[1], existing_caption.split(': ')[1], similarity_threshold)
                   for existing_caption in unique_captions):
            unique_captions.append(caption)

    # Join the unique captions into a paragraph
    summary = "This video shows the following sequence of events: "
    summary += " ".join([caption.split(': ')[1] for caption in unique_captions])

    return summary

# Load captions
captions = load_captions('captions.txt')

# Create a detailed paragraph summary without repeated or similar captions
paragraph_summary = create_paragraph_summary(captions)

# Save the detailed paragraph summary to a text file
with open('video_summary.txt', 'w') as f:
    f.write(paragraph_summary)

print("Paragraph Summary Created and saved to video_summary.txt")


Paragraph Summary Created and saved to video_summary.txt


In [ ]:
import os

# Function to load captions from the captions.txt file
def load_captions(captions_file):
    with open(captions_file, 'r') as f:
        captions = f.readlines()
    return [caption.strip() for caption in captions]

# Function to create a detailed summary from all captions
def create_detailed_summary(captions):
    summary = "This video contains the following sequence of events:\n\n"
    for i, caption in enumerate(captions):
        summary += f"{i+1}. {caption.split(': ')[1]}\n"
    return summary

# Load captions
captions = load_captions('captions.txt')

# Create a detailed summary
detailed_summary = create_detailed_summary(captions)

# Save the detailed summary to a text file
with open('video_summary.txt', 'w') as f:
    f.write(detailed_summary)

print("Detailed Summary Created and saved to video_summary.txt")


Detailed Summary Created and saved to video_summary.txt


In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
%%writefile app.py

import streamlit as st

# Video summary from the captions
summary = video_summary.txt

# Function to handle user questions and generate responses based on the summary
def get_response(question, summary):
    # Very basic logic to handle queries
    if any(word in question.lower() for word in ["what", "happening", "event", "going on"]):
        return f"This is a summary of what happens in the video: {summary}"
    elif any(word in question.lower() for word in ["who", "person", "people"]):
        return "Sorry, this chatbot doesn't handle specific people recognition in the video."
    elif "how" in question.lower():
        return "The video appears to show a sequence of events, based on the extracted frames."
    else:
        return "I'm not sure about that, but here's what I know from the video: " + summary

# Streamlit chatbot interface
st.title("Video Summary Chatbot")

#st.header("Summary of the video:")
#st.write(summary)

# Chatbot input section
st.header("Ask about the video")
user_input = st.text_input("Enter your question about the video:")

# Generate response based on the user's input
if user_input:
    response = get_response(user_input, summary)
    st.write("You asked:", user_input)
    st.write("Response:", response)


Overwriting app.py


In [ ]:
!ngrok authtoken 2nUjE0dqrTERAPjNBtZSX29bOYI_6zJ1Z2aTCVm5YckL3hL5y

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Start an HTTP tunnel using ngrok with the correct Streamlit port
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://f382-35-236-246-161.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/dev/null&